In [ ]:
from lxml import etree
import re
import pandas as pd
import multiprocessing
import itertools

def process_element(elem):    
    if elem[1].text == '0':
        text = elem[-1][-2].text
        if text is not None:
            if "#REDIRECT [[" in text:
                return None
            title = elem[0].text
            length = len(text)
            is_dis = "{{disambig}}" in text
            is_stub = ("{{Stub}}" in text) or ("-stub}}" in text)
            return (title, length, is_dis, is_stub )
    return None


def scan(dump):
    print "starting scan of dump %s" % dump
    i = 0
    context = etree.iterparse( dump , tag = '{http://www.mediawiki.org/xml/export-0.10/}page')
    l = []
    for event, elem in context:
        i+=1
        result_tuple = process_element(elem)
        if result_tuple is not None:
            l.append(result_tuple)
        elem.clear()
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
    del context
    print "finishing scan of dump %s" % dump
    
    return l


def scan_dump_parrallel(wikilang, n):
    exp_dir = 'en-%s' % wikilang
    args = ['/home/ellery/%s/dump%d.xml' % (exp_dir, i) for i in range(1, n+1)]
    p = multiprocessing.Pool(8)
    results = p.map(scan, args)
    l = list(itertools.chain(*results))
    df = pd.DataFrame(l)
    df.columns = ['title', 'length', 'is_dis', 'is_stub']
    return df

df = scan_dump_parrallel('en', 27)
df.to_csv('/home/ellery/en-en/dump_features.tsv',  sep = '\t', encoding = 'utf8')
print df.shape

In [4]:
df = pd.read_csv('/home/ellery/en-en/dump_features.tsv',  sep = '\t', encoding = 'utf8')

In [5]:
df.head()

,Unnamed: 0,title,length,is_dis,is_stub
0,0,Anarchism,176778,False,False
1,1,Autism,128968,False,False
2,2,Albedo,34095,False,False
3,3,A,16098,False,False
4,4,Alabama,161935,False,False


In [6]:
df.shape

(5719260, 5)

In [7]:
df['is_dis'].sum()

75433

In [8]:
df['is_stub'].sum()

1894923

In [9]:
df[df['is_stub']][:10]

,Unnamed: 0,title,length,is_dis,is_stub
11,11,Actrius,1747,False,True
90,90,List of anthropologists,6179,False,True
153,153,Analog Brothers,2611,False,True
197,197,American shot,1719,False,True
238,238,Northern cavefish,2408,False,True
263,263,Geography of American Samoa,2964,False,True
267,267,Transportation in American Samoa,1372,False,True
273,273,America's National Game,810,False,True
383,383,Ancylopoda,1693,False,True
535,535,Alexander Aetolus,4052,False,True
